In [44]:
import numpy as np
import pandas as pd

from pathlib import Path
import copy



### Plan:

1. transform data
1. use a logisitc regression model as a a baseline for prediciton of the trainng set
1. use a mulit-layed layered neural network to predict a binary prediction fo survival for he training set

#### compeition docs 
https://www.kaggle.com/competitions/titanic/data

#### data
data dictionary
        Variable	Definition	Key
        survival	- Survival	0 = No, 1 = Yes
        pclass - 	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
        sex	- Sex	
        Age	- Age in years	
        sibsp	- # of siblings / spouses aboard the Titanic	
        parch	- # of parents / children aboard the Titanic	
        ticket	- Ticket number	
        fare	- Passenger fare	
        cabin	- Cabin number	
        embarked	- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

#### activation funciton:
- layer 1 - Relu
- output layer - sigmoid

#### formulas:
- logistic regression:
        sigmoid(w.T x  + b)
- sigmoid:
        np.sigmoid()
- relu:
        np.maximum(arr, 0)
- back propagation:

In [45]:
# load training data

DATA_PATH = Path('/Users/tommadden/Library/CloudStorage/OneDrive-petsathome.onmicrosoft.com/machine_learning_specialisation/titanic_survival/data')
p = Path(DATA_PATH, 'titanic', 'train.csv')
train_df = pd.read_csv(p, index_col=0)

p = Path(DATA_PATH, 'titanic', 'test.csv')
test_df = pd.read_csv(p, index_col=0)


p = Path(DATA_PATH, 'titanic', 'gender_submission.csv')
test_Y = pd.read_csv(p, index_col=0)

train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [46]:
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [47]:
def clean_data(df):

    # transform input parameters
    input_df = df
    
    # sex classification: male = 1, female = 0
    input_df['Sex'] = df.Sex.eq('male').mul(1)

    # fill NaN age to average age (29.5)
    input_df['Age'] = input_df['Age'].fillna(29.5)
    
    input_df.Cabin.fillna(0, inplace = True)
    # set records with multiple cabin assignments to first cabin assingment (assumes this will not introduce bias)
    input_df['CabinLevel'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.split().values]
    # regex to isolate first letter of cabin name (presumed to be the level of the ship)
    input_df['CabinLevel'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.findall(r'[A-Z]').values]

    # cabin counts to columns
    input_df['CabinLevel'].fillna(0, inplace = True)
    levels = input_df['CabinLevel'].unique()
    for i in levels:
        input_df[str('CabinLevel'+str(i))] = np.where(input_df['CabinLevel'] == i, 1, 0)

    # parse and store cabin number from cabin cloumn into canibno cloumn
    input_df['CabinNo'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.split().values]

    num_arr = input_df.CabinNo.str.findall(r'[0-9]+').values

    CabinNoArr = []
    for i in num_arr:
        if type(i) == list:
            try:
              CabinNoArr.append(i[0])  
            except:
                CabinNoArr.append(i)
        else:
            CabinNoArr.append(i)

    input_df['CabinNo'] = CabinNoArr    
    input_df['CabinNo'].fillna(0, inplace=True)

    # include only unwanted comlumns
    filt = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
           'Fare', 'CabinLevel0', 'CabinLevelC',
           'CabinLevelE', 'CabinLevelG', 'CabinLevelD', 'CabinLevelA',
           'CabinLevelB', 'CabinLevelF']
    input_df = input_df[filt]

    # exclude unwanted columns
#     input_df = input_df.loc[:, ~input_df.columns.isin(['Survived','Name','Ticket', 'Embarked','Cabin','CabinLevel'])]

    return input_df

clean_train_df = clean_data(train_df)
clean_train_Y = train_df['Survived']
clean_test_df = clean_data(test_df)


In [48]:
test_df.columns, train_df.columns

(Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
        'Cabin', 'Embarked', 'CabinLevel', 'CabinLevel0', 'CabinLevelB',
        'CabinLevelE', 'CabinLevelA', 'CabinLevelC', 'CabinLevelD',
        'CabinLevelF', 'CabinLevelG', 'CabinNo'],
       dtype='object'),
 Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
        'Fare', 'Cabin', 'Embarked', 'CabinLevel', 'CabinLevel0', 'CabinLevelC',
        'CabinLevelE', 'CabinLevelG', 'CabinLevelD', 'CabinLevelA',
        'CabinLevelB', 'CabinLevelF', 'CabinLevelT', 'CabinNo'],
       dtype='object'))

In [49]:
def df_to_matrix(matrix, labels):
    
    # transform df variables to matrix

    m = matrix.shape[0]
    n = matrix.shape[1]

    # pandas df to matrix & transpose so that there are 16 features and 891 training examples
    # each column is a single training example, each column is a feature
    matrix = matrix.values.T
    labels = labels.values.reshape(1,m)

    assert matrix.shape == (n,m) 
    assert labels.shape == (1,m)

    print(matrix.shape, labels.shape)
    
    return matrix, labels

In [50]:
# transform variables to matrix
X_train, Y_train = df_to_matrix(clean_train_df, clean_train_Y)

X_test, Y_test = df_to_matrix(clean_test_df, test_Y )


(14, 891) (1, 891)
(14, 418) (1, 418)


In [51]:
# notes
arr = np.array([i * 0.01 for i in range(-10,10)])
# tanh fucntion
np.tanh(arr)

# relu function
np.maximum(arr,0)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])

In [52]:
# deep learning specialisatoin logistic regression helper functions

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1 / (1+np.exp(1)**-z)
    return s



def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias) of type float
    """
    
    w = np.zeros(dim).reshape(dim,1)
    b = 0.0

    return w, b



def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]

    # forward propagation
    A = sigmoid(np.dot(w.T.astype(float), X.astype(float)) + b)
    cost = - np.mean(Y * np.log(A) + (1 - Y ) * np.log(1 - A))
    
    # back propagation
    dw = 1/m * np.dot(X, (A-Y).T)
    db = np.mean(A-Y)
        
    cost = np.squeeze(np.array(cost))
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost



def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)
                
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule 
        w = w - learning_rate * dw
        b = b - learning_rate * db

        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
            # Print the cost every 100 training iterations
            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs



def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of yhat = y
    
    A = sigmoid(np.dot(w.T, X) + b) 
        
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[0,i] >= 0.5 :
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
                    
    return Y_prediction



def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to True to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    # initialize parameters with zeros 
    w, b = initialize_with_zeros(X_train.shape[0])

    # calculates loss vs training set, calculates derivates and updates w & b
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # updates w,b to optisied values
    w = params['w']
    b = params['b']
    
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
 
    # Print train/test Errors
    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [65]:
# implement model

log_model = model(X_train, Y_train, X_test, Y_test, num_iterations=10000, learning_rate=0.5, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 100: nan
Cost after iteration 200: nan
Cost after iteration 300: nan
Cost after iteration 400: nan
Cost after iteration 500: nan
Cost after iteration 600: nan
Cost after iteration 700: nan
Cost after iteration 800: nan
Cost after iteration 900: nan
Cost after iteration 1000: nan
Cost after iteration 1100: nan
Cost after iteration 1200: nan
Cost after iteration 1300: nan
Cost after iteration 1400: nan
Cost after iteration 1500: nan
Cost after iteration 1600: nan
Cost after iteration 1700: nan
Cost after iteration 1800: nan
Cost after iteration 1900: nan


/var/folders/xg/mm75c5z52274c3ylzwr2jg200000gp/T/ipykernel_914/4240751090.py:60: RuntimeWarning: divide by zero encountered in log
  cost = - np.mean(Y * np.log(A) + (1 - Y ) * np.log(1 - A))
/var/folders/xg/mm75c5z52274c3ylzwr2jg200000gp/T/ipykernel_914/4240751090.py:60: RuntimeWarning: invalid value encountered in multiply
  cost = - np.mean(Y * np.log(A) + (1 - Y ) * np.log(1 - A))
/var/folders/xg/mm75c5z52274c3ylzwr2jg200000gp/T/ipykernel_914/4240751090.py:13: RuntimeWarning: overflow encountered in power
  s = 1 / (1+np.exp(1)**-z)


Cost after iteration 2000: nan
Cost after iteration 2100: nan
Cost after iteration 2200: nan
Cost after iteration 2300: nan
Cost after iteration 2400: nan
Cost after iteration 2500: nan
Cost after iteration 2600: nan
Cost after iteration 2700: nan
Cost after iteration 2800: nan
Cost after iteration 2900: nan
Cost after iteration 3000: nan
Cost after iteration 3100: nan
Cost after iteration 3200: nan
Cost after iteration 3300: nan
Cost after iteration 3400: nan
Cost after iteration 3500: nan
Cost after iteration 3600: nan
Cost after iteration 3700: nan
Cost after iteration 3800: nan
Cost after iteration 3900: nan
Cost after iteration 4000: nan
Cost after iteration 4100: nan
Cost after iteration 4200: nan
Cost after iteration 4300: nan
Cost after iteration 4400: nan
Cost after iteration 4500: nan
Cost after iteration 4600: nan
Cost after iteration 4700: nan
Cost after iteration 4800: nan
Cost after iteration 4900: nan
Cost after iteration 5000: nan
Cost after iteration 5100: nan
Cost aft

In [60]:
log_model

{'costs': [array(0.69314718)],
 'Y_prediction_test': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
         1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,

In [55]:
X_exp = X_test.T[0:6].T
X_exp

array([[ 3.    ,  3.    ,  2.    ,  3.    ,  3.    ,  3.    ],
       [ 1.    ,  0.    ,  1.    ,  1.    ,  0.    ,  1.    ],
       [34.5   , 47.    , 62.    , 27.    , 22.    , 14.    ],
       [ 0.    ,  1.    ,  0.    ,  0.    ,  1.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  1.    ,  0.    ],
       [ 7.8292,  7.    ,  9.6875,  8.6625, 12.2875,  9.225 ],
       [ 1.    ,  1.    ,  1.    ,  1.    ,  1.    ,  1.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

In [56]:
Y_exp = Y_test.T[0:6]
Y_exp

array([[0],
       [1],
       [0],
       [0],
       [1],
       [0]])

In [57]:
w = log_model['w']
b = log_model['b']

# X_exp, Y_exp, w, b

predict(w,b, X_exp)

array([[0., 0., 0., 0., 1., 0.]])